<a href="https://colab.research.google.com/github/jiho051225/footballdata6week/blob/main/intercept/intercept_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/narame7/UOS-FootballDataAnalytics-Tutorial

In [ ]:
!pip install pandera==0.13.4 scipy==1.11.4 statsbombpy

In [ ]:
!pip install --upgrade pip
!pip install multimethod==1.9
!pip install pandera==0.13.4
!pip install git+https://github.com/ML-KULeuven/socceraction.git
!pip install matplotsoccer

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")

from tqdm.notebook import tqdm

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import matplotsoccer as mps # 축구 이벤트 데이터 시각화
from socceraction.data.statsbomb import StatsBombLoader # StatsBomb 데이터 로더
import socceraction.spadl as spadl # SPADL 포맷 변환 및 처리
import socceraction.xthreat as xthreat # Expected Threat (xT) 계산

### Set up the StatsBombLoader

In [ ]:
# StatsBomb 데이터 로더 객체를 생성합니다. 무료 공개 데이터 접근을 위해 creds를 None으로 설정합니다.
SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
SBL

In [ ]:
# 사용 가능한 대회 목록
competitions = SBL.competitions()
competitions

In [ ]:
# 'FIFA World Cup' 2018 시즌
selected_competitions = competitions[
	(competitions.competition_name == "FIFA World Cup")
	& (competitions.season_name == "2018")
]
selected_competitions

In [ ]:
games = pd.concat([
	SBL.games(row.competition_id, row.season_id)
	for row in selected_competitions.itertuples()
])
games[["home_team_id", "away_team_id", "game_date", "home_score", "away_score"]]

### 🥅 SPADL (Soccer Player Action Description Language) 전처리 및 로딩

- SPADL: StatsBomb, Opta, Wyscout 등 다양한 데이터 제공 업체의 서로 다른 이벤트 데이터 포맷을 통일된 액션 기반 포맷으로 변환하는 언어입니다.
- 현대 축구 데이터 분석 연구에서는 SPADL이라는 통일된 포맷을 사용하여 분석하는 연구가 주를 이룹니다.
- 주요 정보: 액션 유형, 시작/종료 좌표, 선수, 팀, 결과 등
- [논문링크](https://dl.acm.org/doi/10.1145/3292500.3330758)
- [Github](https://github.com/ML-KULeuven/socceraction/tree/master)

In [ ]:
# 모든 액션이 왼쪽에서 오른쪽으로 진행되도록 좌표를 변환하는 함수입니다.
# xT 모델은 공격 방향을 통일해야 합니다.
def play_left_to_right(actions: pd.DataFrame, home_team_id: int) -> pd.DataFrame:
	"""
		모든 액션이 동일한 경기 방향(왼쪽에서 오른쪽)으로 수행되도록 변환합니다.

		이 함수는 각 액션의 시작 및 종료 위치를 변경하여,
		마치 첫 액션을 수행한 팀이 항상 왼쪽에서 오른쪽으로 플레이하는 것처럼 만듭니다.

		Parameters
		----------
		actions : pd.DataFrame
			한 경기의 액션 데이터프레임.
		home_team_id : int
			홈팀의 ID.

		Returns
		-------
		pd.DataFrame
			모든 액션이 왼쪽에서 오른쪽으로 수행되도록 좌표가 변환된 데이터프레임.
	"""

	away_idx = actions.team_id != home_team_id

	for col in ["start_x", "end_x"]:
		actions.loc[away_idx, col] = spadl.config.field_length - actions[away_idx][col].values
	for col in ["start_y", "end_y"]:
		actions.loc[away_idx, col] = spadl.config.field_width - actions[away_idx][col].values

	return actions

In [ ]:
games_verbose = tqdm(list(games.itertuples()), desc="Loading game data")
df_teams, df_players, df_actions = [], [], []

for game in games_verbose:
	# load data
	df_teams.append(SBL.teams(game.game_id))
	df_players.append(SBL.players(game.game_id))
	events = SBL.events(game.game_id)

	# StatsBomb 이벤트 데이터를 SPADL 액션 데이터로 변환합니다.
	actions = spadl.statsbomb.convert_to_actions(
			events,
			home_team_id=game.home_team_id,
			xy_fidelity_version=1,
			shot_fidelity_version=1
	)
	actions = play_left_to_right(actions, home_team_id=game.home_team_id) # xT 계산을 위해 모든 액션 방향을 왼쪽에서 오른쪽으로 통일합니다.
	df_actions.append(actions)

df_teams = pd.concat(df_teams).drop_duplicates(subset="team_id")
df_player_games = pd.concat(df_players).reset_index(drop=True) # 경기 별 선수 정보. 경기 별 선수 정보 (position, minutes_played)은 다름)
df_players = pd.concat(df_players).drop_duplicates(subset="player_id") # 선수 정보. 선수 정보 (name, nationality 등)은 동일)
df_actions = pd.concat(df_actions).reset_index(drop=True)
df_actions.head()

In [ ]:
df_teams

In [ ]:
df_player_games

In [ ]:
df_players

# Intercept Model

In [ ]:
#intercept된 이벤트 데이터
df_actions["type_name"] = df_actions.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
intercept_actions = df_actions[df_actions['type_name'] == 'interception']
print(f"Number of interception actions: {len(intercept_actions)}")
intercept_actions.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the 2D histogram (density) of interception starting positions
# Using 12 bins for y-axis (field width) and 16 bins for x-axis (field length) to match xT grid dimensions
hist_density, x_edges, y_edges = np.histogram2d(
    intercept_actions.start_y,
    intercept_actions.start_x,
    bins=[12, 16],
    range=[[0, spadl.config.field_width], [0, spadl.config.field_length]]
)

# Visualize the density heatmap
mps.heatmap(
    hist_density,
    cmap="hot",
    linecolor="white",
    cbar=True
)

### SPADL 데이터 시각화

- [matplotsoccer](https://github.com/TomDecroos/matplotsoccer/tree/master): 축구 이벤트/액션 데이터를 경기장 위에 시각화하는 라이브러리.

In [ ]:
def nice_time(row):
	minute = int((row.period_id-1)*45 +row.time_seconds // 60)
	second = int(row.time_seconds % 60)
	return f"{minute}m{second}s"

def plot_actions(a: pd.DataFrame, g: pd.Series) -> None:
	home_team_name = df_teams[df_teams.team_id == g.home_team_id].team_name.values[0]
	away_team_name = df_teams[df_teams.team_id == g.away_team_id].team_name.values[0]

	minute = int((a.period_id.values[0]-1) * 45 + a.time_seconds.values[0] // 60)
	game_info = f"{g.game_date} {home_team_name} {g.home_score}-{g.away_score} {away_team_name} {minute + 1}'"
	print(game_info)

	a["player_name"] = a.player_id.map(df_players.set_index("player_id").player_name)
	a["team_name"] = a.team_id.map(df_teams.set_index("team_id").team_name)
	a["type_name"] = a.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
	a["result_name"] = a.result_id.map(spadl.config.results_df().result_name.to_dict())
	a["nice_time"] = a.apply(nice_time, axis=1)

	if "xT_value" in a.columns:
		labels = a[["nice_time", "type_name", "player_name", "team_name", "xT_value"]]
		labeltitle = ["time", "actiontype", "player", "team", "xT_value"]
	else:
		labels = a[["nice_time", "type_name", "player_name", "team_name"]]
		labeltitle = ["time", "actiontype", "player", "team"]

	away_idx = a.team_id != g.home_team_id
	for col in ["start_x", "end_x"]:
		a.loc[away_idx, col] = spadl.config.field_length - a[away_idx][col].values
	for col in ["start_y", "end_y"]:
		a.loc[away_idx, col] = spadl.config.field_width - a[away_idx][col].values

	mps.actions(
		location=a[["start_x", "start_y", "end_x", "end_y"]],
		action_type=a.type_name,
		team= a.team_name,
		result= a.result_name == "success",
		label=labels,
		labeltitle=labeltitle,
		zoom=False,
		figsize=6,
		color="green"
	)

In [ ]:
# Select the 5 actions preceding the 2-0
shot = 5
a = df_actions[shot-5:shot+2].copy()

# Print the game date and timestamp of the goal
game_id = 7559
g = games[games.game_id == game_id].iloc[0]

plot_actions(a, g)
a

### Expected Threat (xT)


- [Expected Threat 참고 자료](https://karun.in/blog/expected-threat.html)
- xT 계산에 사용되는 액션: pass(cross), dribble, shot(goal)

In [ ]:
df_actions

In [ ]:
A = df_actions.copy()
A["type_name"] = A.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
A["result_name"] = A.result_id.map(spadl.config.results_df().result_name.to_dict())
A["bodypart_name"] = A.bodypart_id.map(spadl.config.bodyparts_df().bodypart_name.to_dict())

A.head()

In [ ]:
A["type_name"].value_counts()

In [ ]:
A[A["type_name"].isin(["pass", "cross", "dribble", "shot"])][["type_name", "result_name"]].value_counts().sort_index()

### xT Model 학습

In [ ]:
# 사전 학습된 xT 모델을 로드할 수도 있습니다.
url_grid = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xTModel = xthreat.load_model(url_grid)

In [ ]:
print(f"xT shape: {xTModel.xT.shape}")
xTModel.xT

In [ ]:
# 16x12 그리드로 xT 모델을 초기화하고 학습시킵니다
xTModel = xthreat.ExpectedThreat(l=16, w=12)
xTModel.fit(A)

In [ ]:
print(f"xT shape: {xTModel.xT.shape}")
xTModel.xT

In [ ]:
# 학습된 xT 값을 히트맵으로 시각화.
mps.heatmap(xTModel.xT, cmap="hot", linecolor="white", cbar="True")


# 그리드 셀 사이의 값을 부드럽게 보간(interpolate)하여 경기장 내 모든의 좌표 (x, y)에서의 xT 값을 추정.
# 경기장 전체 크기(105x68)에 맞춰 세밀한 좌표 그리드를 생성합니다. (105 x 68)
interp = xTModel.interpolator()
x = np.linspace(0, 105, 1050)
y = np.linspace(0, 68, 680)
mps.heatmap(interp(x,y), cmap="hot", linecolor="white", cbar="True")

In [ ]:
# xT 값은 공을 성공적으로 이동시키고 소유권을 유지한 액션에 대해서만 평가.
mov_actions = xthreat.get_successful_move_actions(A)

# rate: xT_value = xT(end_location) - xT(start_location)
mov_actions["xT_value"] = xTModel.rate(mov_actions)
mov_actions[["type_name", "start_x", "start_y", "end_x", "end_y", "xT_value"]][:10]

In [ ]:
# xT 값 분포를 시각화하여 액션들의 가치 평가 경향을 확인합니다.
a = mov_actions.copy()

# Show scatterplot of actions colored according to xT value
mps.field(show=False)
plt.title("Actions colored according to xT value")
plt.scatter(a.start_x, a.start_y, c=a.xT_value, cmap="bwr_r")
plt.colorbar()
plt.show()

# xT 값이 낮은 액션
a = a.sort_values("xT_value", ascending=True)
mps.field(show=False)
plt.scatter(a.start_x, a.start_y, c=a.xT_value, cmap="bwr_r")
plt.colorbar()
plt.show()

# xT 값이 높은 액션
a = a.sort_values("xT_value", ascending=False)
mps.field(show=False)
plt.scatter(a.start_x, a.start_y, c=a.xT_value, cmap="bwr_r")
plt.colorbar()
plt.show()

In [ ]:
# xT 모델 학습 과정(Value Iteration)의 각 단계별 xT 값 변화를 3D Surface 플롯으로 시각화하는 함수
def visualize_surface_plots(xTModel):
	"""Visualizes the surface plot of each iteration of the model.

	See https://plot.ly/python/sliders/ and https://karun.in/blog/expected-threat.html#visualizing-xt
	NOTE: y-axis is mirrored in plotly.
	"""
	camera = dict(
		up=dict(x=0, y=0, z=1),
		center=dict(x=0, y=0, z=0),
		eye=dict(x=-2.25, y=-1, z=0.5),
	)

	max_z = np.around(xTModel.xT.max() + 0.05, decimals=1)

	layout = go.Layout(
		title="Expected Threat",
		autosize=True,
		width=500,
		height=500,
		margin=dict(l=65, r=50, b=65, t=90),
		scene=dict(
			camera=camera,
			aspectmode="auto",
			xaxis=dict(),
			yaxis=dict(),
			zaxis=dict(autorange=False, range=[0, max_z]),
		),
	)

	fig = go.Figure(layout=layout)

	for i in xTModel.heatmaps:
		fig.add_trace(go.Surface(z=i))

	# Make last trace visible
	for i in range(len(fig.data) - 1):
		fig.data[i].visible = False
	fig.data[len(fig.data) - 1].visible = True

	# Create and add slider
	steps = []
	for i in range(len(fig.data)):
		step = dict(method="restyle", args=["visible", [False] * len(fig.data)])
		step["args"][1][i] = True  # Toggle i'th trace to "visible"
		steps.append(step)

	sliders = [
		dict(
			active=(len(fig.data) - 1),
			currentvalue={"prefix": "Iteration: "},
			pad={"t": 50},
			steps=steps,
		)
	]

	fig.update_layout(sliders=sliders)
	fig.show()

visualize_surface_plots(xTModel)

### 선수별 xT 평가

In [ ]:
df_player_games

In [ ]:
df_player_info = df_player_games.groupby("player_id").agg(
	total_minutes_played=("minutes_played", "sum"),
	num_games=("game_id", "nunique"),
	main_position=("starting_position_name", lambda x: x.mode()[0])
).reset_index(drop=False)
df_player_info

In [ ]:
mov_actions

In [ ]:
# Compute each player's number of actions and total xT values
mov_actions["xT_count"] = 1

playersR = (
    mov_actions[["player_id", "xT_value", "xT_count"]]
    .groupby(["player_id"])
    .sum()
    .reset_index()
)

playersR["player_name"] = playersR.player_id.map(df_players.set_index("player_id").player_name)
playersR["team_id"] = playersR.player_id.map(df_players.set_index("player_id").team_id)
playersR["team_name"] = playersR.team_id.map(df_teams.set_index("team_id").team_name)
playersR.sort_values("xT_value", ascending=False).head(10)

In [ ]:
print(f"평균 출전 시간: {df_player_info.total_minutes_played.mean()}")
df_player_info.total_minutes_played.describe()

In [ ]:
# Normalize for minutes played

playersR["total_minutes_played"] = playersR.player_id.map(df_player_info.set_index("player_id").total_minutes_played)
playersR["xT_per_90"] = playersR.xT_value * 90 / playersR.total_minutes_played
playersR[playersR.total_minutes_played > df_player_info.total_minutes_played.mean()].sort_values("xT_per_90", ascending=False).head(10) # at least 235 minutes played